In [1]:
"""Differential Expression and CLuster Annotation.
The tasks of this project include the following
- Look for Upregulation of marker genes for cell types of interest
- Compare the complete gene expression profiles between groups
-Use automated methods to compare cells of interest to databases of cell 
type expression profiles to combine clustering and annotation"""





'Differential Expression and CLuster Annotation.\nThe tasks of this project include the following\n- Look for Upregulation of marker genes for cell types of interest\n- Compare the complete gene expression profiles between groups\n-Use automated methods to compare cells of interest to databases of cell \ntype expression profiles to combine clustering and annotation'

In [2]:
#Packages installation

import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from matplotlib import pyplot as plt

# to make matplotlib display inline in the jupyter notebook
%matplotlib inline 


In [3]:
#Loading Data file
bdata = sc.read('test_adata.h5ad')

In [4]:
bdata

AnnData object with n_obs × n_vars = 30474 × 13553
    obs: 'batch_indices', 'n_genes', 'percent_mito', 'leiden_subclusters', 'cell_types', 'tissue', 'batch'
    obsm: 'isotypes_htos', 'protein_expression'

In [5]:
print(bdata.obs.shape)
bdata.obs.head()
#obtains top observation data across each cell

(30474, 7)


,batch_indices,n_genes,percent_mito,leiden_subclusters,cell_types,tissue,batch
index,,,,,,,
AAACCCAAGGGTAATT-1,0,3137,0.062138,"12,0",NKT,Spleen,SLN111-D1
AAACCCAAGGTAAACT-1,0,2256,0.057545,6,CD122+ CD8 T,Spleen,SLN111-D1
AAACCCACACTAGGTT-1,0,1367,0.058373,3,Transitional B,Spleen,SLN111-D1
AAACCCACAGATACCT-1,0,1567,0.065386,4,Mature B,Lymph_Node,SLN111-D1
AAACCCACAGGAATAT-1,0,1895,0.059644,0,CD4 T,Lymph_Node,SLN111-D1


In [6]:
#getting more details about bdata
print(bdata.var.shape)
bdata.var.head()

#bdata.var was used to retreive information about genes present in data set

(13553, 0)


""
index
Mrpl15
Lypla1
Tcea1
Atp6v1h
Rb1cc1


In [7]:
bdata.var_names  #names of all genes present

Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Atp6v1h', 'Rb1cc1', '4732440D04Rik',
       'Pcmtd1', 'Gm26901', 'Rrs1', 'Adhfe1',
       ...
       'Tmlhe', 'AC133103.1', 'AC132444.1', 'Csprs', 'AC132444.6',
       'AC125149.3', 'AC168977.1', 'PISD', 'DHRSX', 'CAAA01147332.1'],
      dtype='object', name='index', length=13553)

In [8]:
bdata.X #create numpy array of Cells for X number of genes

<30474x13553 sparse matrix of type '<class 'numpy.float32'>'
	with 43762899 stored elements in Compressed Sparse Row format>

In [12]:
#Creating AnnData from csv files in dataset

count_dataframe = pd.read_csv('brain_counts.csv', index_col=0) #use first columns to label rows


#print first two rows of data frame

count_dataframe.head(2)

,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007N19Rik,0610007P08Rik,0610007P14Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,0610009B22Rik,...,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,zsGreen_transgene
A1.B003290.3_38_F.1.1,0,125,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,54,0
A1.B003728.3_56_F.1.1,0,0,0,0,0,324,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#shape of count data frame indicating number of cells and number of genes
print(count_dataframe.shape)

(3401, 23433)


In [14]:
""" BRAIN METADATA"""

metadata_dataframe = pd.read_csv('brain_metadata.csv', index_col=0)

print(metadata_dataframe.shape)
metadata_dataframe.head(2)

#meta data shows 5 columns of informatino about 3,401 cells

(3401, 5)


,cell_ontology_class,subtissue,mouse.sex,mouse.id,plate.barcode
cell,,,,,
A1.B003290.3_38_F.1.1,astrocyte,Striatum,F,3_38_F,B003290
A1.B003728.3_56_F.1.1,astrocyte,Striatum,F,3_56_F,B003728


In [15]:
#counting the number of times each value appears in a column like the following:
print(pd.value_counts(metadata_dataframe['subtissue']))


Cortex         1149
Hippocampus     976
Striatum        723
Cerebellum      553
Name: subtissue, dtype: int64


In [19]:
"""CONSTRUCTING AnnData USING two CSV files"""

adata = sc.AnnData(X= count_dataframe, obs= metadata_dataframe)

print(adata.shape)

adata.X #expression matrix


C:\Users\samen\AppData\Local\Temp/ipykernel_14536/3732495509.py:3: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(X= count_dataframe, obs= metadata_dataframe)


(3401, 23433)


array([[  0., 125.,  16., ...,   0.,  54.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0., 348.,   0., ...,   0., 113.,   0.],
       ...,
       [  0., 135.,  41., ...,   0.,  45.,   0.],
       [  0., 129.,  54., ...,   0.,  57.,   0.],
       [  0.,   1.,   0., ...,   0.,   0.,   0.]], dtype=float32)

"""Labelling Spike-ins
Because this is smartseq2 data, we may have spike-ins. An RNA spike-in is an RNA transcript of known sequence and quantity used to calibrate measurements in RNA hybridization assays, such as DNA microarray experiments, RT-qPCR, and RNA-Seq.

A spike-in is designed to bind to a DNA molecule with a matching sequence, known as a control probe. This process of specific binding is called hybridization. A known quantity of RNA spike-in is mixed with the experiment sample during preparation. The degree of hybridization between the spike-ins and the control probes is used to normalize the hybridization measurements of the sample RNA.

These gene names start with ERCC. We can label them in adata.var as a gene annotation. """

In [20]:
is_spike_in = {}
number_of_spike_ins = 0

for gene_name in adata.var_names:
    if 'ERCC' in gene_name:
        is_spike_in[gene_name] = True #record that we found a spike-in
        number_of_spike_ins += 1 #increase the counter
    else: 
        is_spike_in[gene_name] = False #record that  isn't a spike-in

adata.var['ERCC'] = pd.Series(is_spike_in)# because the index of adata.var and they keys of is_spike_in match, anndata will take care of matching them up
print('found this many spike ins: ', number_of_spike_ins)

#This code finds all the spike-ins, counted them as True in the gene info.var and counted non spike ins as false

found this many spike ins:  92


In [21]:
adata.var.head()

,ERCC
0610005C13Rik,False
0610007C21Rik,False
0610007L01Rik,False
0610007N19Rik,False
0610007P08Rik,False


In [22]:
#Saving the AnnData object later use
adata.write('brain_raw.h5ad') #the h5ad extension is AnnData-specific